In [1]:
def reproduceResult():
  seed_value= 0

  
  with tf.device("/cpu:0"):
    ...


  os.environ['PYTHONHASHSEED']=str(seed_value)
  np.random.seed(0)
  rn.seed(0)


  session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, 
                                          inter_op_parallelism_threads=1)


  tf.compat.v1.set_random_seed(seed_value)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
  tf.compat.v1.keras.backend.set_session(sess)
  tf.compat.v1.keras.backend.clear_session()

  


In [2]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
  
import os 
import numpy as np
import tensorflow as tf
import random as rn
from tensorflow import keras

reproduceResult()
# %tensorflow_version 2.x
# import tensorflow as tf
# tf.test.gpu_device_name()
# from scipy import integrate
# import os
# import numpy as np
# from tensorflow import keras
import tempfile
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
# import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
import tqdm

from tensorflow.keras import callbacks

from keras_lr_finder import LRFinder
from clr.clr_callback import CyclicLR

import tensorflow_model_optimization as tfmot

import gensim

import keras_tuner
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters
from attention import Attention

C:\Users\moshi\AppData\Local\Temp/ipykernel_13612/3117544919.py:43: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [3]:
temp = pd.read_csv('Twitter_Emotion_Dataset.csv')
train, test = train_test_split(temp, test_size=0.2, stratify = temp['label'], random_state = 42)
num_classes = 5
embed_num_dims = 300
max_seq_len = 50

x_train = train['tweet']
x_test = test['tweet']

y_train = train['label']
y_test = test['label']

texts_train = x_train
texts_test = x_test

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['tweet'])

sequence_train = tokenizer.texts_to_sequences(texts_train)
sequence_test = tokenizer.texts_to_sequences(texts_test)

index_of_words = tokenizer.word_index

vocab_size = len(index_of_words) + 1

print('Number of unique words: {}'.format(len(index_of_words)))

X_train_pad = pad_sequences(sequence_train, maxlen = max_seq_len, padding='pre' )
X_test_pad = pad_sequences(sequence_test, maxlen = max_seq_len,  padding='pre')

print(X_train_pad)


encoding = {
    'anger': 0,
    'happy': 1,
    'sadness': 2,
    'fear': 3,
    'love': 4
}


y_train = [encoding[x] for x in train['label']]
y_test = [encoding[x] for x in test['label']]



y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)





Number of unique words: 17025
[[    0     0     0 ...     2  7061  4466]
 [    0     0     0 ...     3  4468  7064]
 [    0     0     0 ...  4470  4470  4471]
 ...
 [    0     0     0 ...  3511   263    17]
 [    0     0     0 ...  3930    77 17025]
 [    0     0     0 ...    58   445  1465]]


In [4]:
all_training_words = [word for tokens in train["tweet"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in train["tweet"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

640559 words total, with a vocabulary size of 94
Max sentence length is 500


In [5]:
all_test_words = [word for tokens in test["tweet"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in test["tweet"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

161973 words total, with a vocabulary size of 93
Max sentence length is 285


In [6]:
word2vec_path = 'GoogleNews-vectors-negative300.bin'
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
    if len(tokens_list)<1:
        return np.zeros(k)
    if generate_missing:
        vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
    else:
        vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]
    length = len(vectorized)
    summed = np.sum(vectorized, axis=0)
    averaged = np.divide(summed, length)
    return averaged

def get_word2vec_embeddings(vectors, clean_comments, generate_missing=False):
    embeddings = clean_comments['tweet'].apply(lambda x: get_average_word2vec(x, vectors, 
                                                                                generate_missing=generate_missing))
    return list(embeddings)

training_embeddings = get_word2vec_embeddings(word2vec, train, generate_missing=True)

In [7]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)
tokenizer.fit_on_texts(train["tweet"].tolist())
training_sequences = tokenizer.texts_to_sequences(train["tweet"].tolist())

train_word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(train_word_index))

Found 17025 unique tokens.


In [8]:
train_cnn_data = pad_sequences(training_sequences, maxlen=max_seq_len )

In [9]:
train_embedding_weights = np.zeros((len(train_word_index)+1, embed_num_dims))
for word,index in train_word_index.items():
    train_embedding_weights[index,:] = word2vec[word] if word in word2vec else np.random.rand(embed_num_dims)
embedd_matrix = train_embedding_weights
print(embedd_matrix.shape)

(17026, 300)


# Random Search


In [10]:
from tensorflow.keras import regularizers
import time
LOG_DIR = f"{int(time.time())}"
seed_value= 0



def build_model(hp):
  
  reproduceResult()

  print('Ya it comes here')
  unit_attention = hp.Int("attention_unit",min_value =32, max_value = 128, step = 16)
  fake_val = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_unit = hp.Int("cnn_1_unit",min_value =16, max_value = 96, step = 16)
  cnn_1_dropout = hp.Float("cnn_1_dropout",min_value = 0.1,max_value = 0.3,step = 0.1)

  lstm_unit = hp.Int("lstm_unit",min_value =64, max_value = 256, step = 32)
  lstm_dropout = hp.Float("lstm_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)
  cnn_2_unit = hp.Int("cnn_2_unit",min_value =64, max_value = 256, step = 32)
  cnn_2_dropout = hp.Float("cnn_2_dropout",min_value = 0.1,max_value = 0.5,step = 0.1)



  seq_input = keras.layers.Input(shape=(max_seq_len,))

  embedded = keras.layers.Embedding(vocab_size,
                          embed_num_dims,
                          input_length = max_seq_len,
                          weights = [embedd_matrix])(seq_input)

  cnn = keras.layers.Conv1D(cnn_1_unit,3,kernel_regularizer=regularizers.l2(1e-4),
                            bias_regularizer=regularizers.l2(1e-2),
                            activity_regularizer=regularizers.l2(1e-4))(embedded)
  cnn = keras.layers.Activation(activation='relu')(cnn)
  cnn = keras.layers.BatchNormalization()(cnn)
  cnn = keras.layers.Dropout(cnn_1_dropout,seed=seed_value)(cnn)

  attention_vec = keras.layers.TimeDistributed(keras.layers.Dense(unit_attention))(cnn)
  attention_vec = keras.layers.Reshape((48,unit_attention))(attention_vec)
  attention_vec = keras.layers.Activation('relu', name = 'cnn_attention_vec')(attention_vec)
  attention_output = keras.layers.Dot(axes = 1)([cnn, attention_vec])


  lstm =keras.layers.LSTM(lstm_unit, recurrent_regularizer=regularizers.l2(1e-4),
                                                      return_sequences=True,kernel_regularizer=regularizers.l2(1e-4),
                                                      bias_regularizer=regularizers.l2(1e-2),
                                                      activity_regularizer=regularizers.l2(1e-4),input_shape =(48,))(attention_output)
  lstm = keras.layers.Activation(activation='relu')(lstm)
  lstm = keras.layers.BatchNormalization()(lstm)
  lstm = keras.layers.Dropout(lstm_dropout,seed=seed_value)(lstm)
  
  

  cnn_2 = keras.layers.Conv1D(cnn_2_unit,3,kernel_regularizer=regularizers.l2(1e-4),
                            bias_regularizer=regularizers.l2(1e-2),
                            activity_regularizer=regularizers.l2(1e-4))(lstm)
  cnn_2 = keras.layers.Activation(activation='relu')(cnn_2)
  cnn_2 = keras.layers.BatchNormalization()(cnn_2)
  cnn_2 = keras.layers.Dropout(cnn_2_dropout,seed=seed_value)(cnn_2)

  max_pooling = keras.layers.GlobalMaxPooling1D()(cnn_2)
  output = keras.layers.Dense(num_classes, activation='softmax')(max_pooling)

  model = keras.Model(inputs = [seq_input], outputs = output)
  model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

  return model


stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                              patience=4,
                              restore_best_weights=True,
                              verbose=0, mode='max')


clr_step_size = int((len(X_train_pad)/64))
base_lr = 1e-3
max_lr = 6e-3
mode = 'exp_range'


clr = CyclicLR(base_lr = base_lr, max_lr = max_lr, step_size = clr_step_size, mode = mode)



tuner = RandomSearch(
    build_model,
    objective = keras_tuner.Objective('val_accuracy', direction="max"),
    max_trials = 50,
    executions_per_trial = 1,
    directory = LOG_DIR
    )
  
tuner.search(x=X_train_pad,y = y_train,epochs = 20, batch_size = 128,callbacks = [stop,clr], 
             validation_data = (X_test_pad,y_test))


tuner.results_summary()

Trial 50 Complete [00h 01m 06s]
val_accuracy: 0.2610669732093811

Best val_accuracy So Far: 0.5698070526123047
Total elapsed time: 01h 34m 51s
INFO:tensorflow:Oracle triggered exit
Results summary
Results in 1647103459\untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
attention_unit: 64
cnn_1_unit: 64
cnn_1_dropout: 0.1
lstm_unit: 192
lstm_dropout: 0.30000000000000004
cnn_2_unit: 192
cnn_2_dropout: 0.2
Score: 0.5698070526123047
Trial summary
Hyperparameters:
attention_unit: 112
cnn_1_unit: 16
cnn_1_dropout: 0.30000000000000004
lstm_unit: 256
lstm_dropout: 0.4
cnn_2_unit: 256
cnn_2_dropout: 0.1
Score: 0.5641316771507263
Trial summary
Hyperparameters:
attention_unit: 32
cnn_1_unit: 32
cnn_1_dropout: 0.30000000000000004
lstm_unit: 256
lstm_dropout: 0.2
cnn_2_unit: 64
cnn_2_dropout: 0.1
Score: 0.5516458749771118
Trial summary
Hyperparameters:
attention_unit: 48
cnn_1_unit: 32
cnn_1_dropout: 0.2
lstm_unit: 224
lstm_dropout